# Serial Read Example #1

In [ ]:
# Example #1
# Source: https://stackoverflow.com/questions/4417546/constantly-print-subprocess-output-while-process-is-running

import subprocess

def execute(cmd):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line 
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

# Example
# for path in execute(["locate", "a"]):
#     print(path, end="")


In [ ]:
# Testing Example 1

cmd = ["xxd", "/dev/ttyUSB0"]

for line in execute(cmd):
    print(line, end="")

## Issues:

- No stdout output shown !


# Serial Read Example #2 


In [1]:
# Example 2

import shlex
import subprocess

def get_serial_output(command):
    process = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc


In [2]:
#cmd = ["xxd", "/dev/ttyUSB0"]
cmd = "xxd /dev/ttyUSB0"

get_serial_output(cmd)

b'00000000: 5359 8003 0001 0131 5443 5359 0707 0001  SY.....1TCSY....'
b'00000010: 01bc 5443 5359 8004 0002 0083 b554 4353  ..TCSY.......TCS'
b'00000020: 5981 0200 0114 4454 4353 5980 0300 0101  Y.....DTCSY.....'
b'00000030: 3154 4353 5980 0500 0600 2a00 7b00 00dc  1TCSY.....*.{...'
b'00000040: 5443 5359 8003 0001 0131 5443 5359 8004  TCSY.....1TCSY..'
b'00000050: 0002 0083 b554 4353 5985 0200 0159 8d54  .....TCSY....Y.T'
b'00000060: 4353 5980 0200 0102 3154 4353 5980 0300  CSY.....1TCSY...'
b'00000070: 0109 3954 4353 5980 0500 0600 3100 7900  ..9TCSY.....1.y.'
b'00000080: 00e1 5443 5359 0101 0001 0fbe 5443 5359  ..TCSY......TCSY'
b'00000090: 8102 0001 1343 5443 5359 8003 0001 0333  .....CTCSY.....3'
b'000000a0: 5443 5359 0707 0001 00bb 5443 5359 8004  TCSY......TCSY..'
b'000000b0: 0002 0083 b554 4353 5980 0300 0106 3654  .....TCSY.....6T'
b'000000c0: 4353 5980 0500 0600 2e00 7a00 00df 5443  CSY.......z...TC'
b'000000d0: 5359 8502 0001 598d 5443 5359 8002 0001  SY....Y.TCSY....'
b'0000

KeyboardInterrupt: 

# Issues:

- Output isnt printed line-by-line, instead displayed in large blocks


# Serial Read Example #3

In [3]:
import io
import selectors
import subprocess
import sys

# Example 3 
# Source: https://gist.github.com/nawatts/e2cdca610463200c12eac2a14efc0bfb

def capture_subprocess_output(subprocess_args):
    # Start subprocess
    # bufsize = 1 means output is line buffered
    # universal_newlines = True is required for line buffering
    process = subprocess.Popen(subprocess_args,
                               bufsize=1,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               universal_newlines=True)

    # Create callback function for process output
    buf = io.StringIO()
    def handle_output(stream, mask):
        # Because the process' output is line buffered, there's only ever one
        # line to read when this function is called
        line = stream.readline()
        buf.write(line)
        sys.stdout.write(line)

    # Register callback for an "available for read" event from subprocess' stdout stream
    selector = selectors.DefaultSelector()
    selector.register(process.stdout, selectors.EVENT_READ, handle_output)

    # Loop until subprocess is terminated
    while process.poll() is None:
        # Wait for events and handle them with their registered callbacks
        events = selector.select()
        for key, mask in events:
            callback = key.data
            callback(key.fileobj, mask)

    # Get process return code
    return_code = process.wait()
    selector.close()

    success = (return_code == 0)

    # Store buffered output
    output = buf.getvalue()
    buf.close()

    return (success, output)


In [4]:
# Example 3 Testing 

# cmd = ["xxd", "/dev/ttyUSB0"]
cmd = ["./getSerialBytes.sh"]


while True:
    code, out = capture_subprocess_output(cmd)
    print(out)


KeyboardInterrupt: 

## Issues:

- still prints serial messages as large blocks and not line-by-line


# Serial Read Example #4: pySerial


In [4]:
import serial
serBarCode = serial.Serial('/dev/ttyUSB0', 115200, timeout=2)

while True:
    #read data from serial port
    dataBarCode = serBarCode.read(10) # read 10 bytes at a time
    #dataBarCode = serBarCode.read(8) # read 8 bytes at a time
    #dataBarCode = serBarCode.readline()
    

    #if there is smth do smth
    if len(dataBarCode) >= 1:
        print(dataBarCode)
        #print(dataBarCode.decode("utf-8"))

b'SY\x80\x03\x00\x01\x011TC'
b'SY\x80\x05\x00\x06\x00(\x00|'
b'\x00\x00\xdbTCSY\x80\x03\x00'
b'\x01\x011TCSY\x80\x04\x00'
b'\x02\x00\x83\xb5TCSY\x85\x02'
b'\x00\x01R\x86TCSY\x80\x03'
b'\x00\x01\x011TCSY\x80\x05'
b'\x00\x06\x00(\x00|\x00\x00\xdbT'
b'CSY\x81\x02\x00\x01\x10@T'
b'CSY\x80\x03\x00\x01\x011T'
b'CSY\x80\x04\x00\x02\x00\x83\xb5'
b'TCSY\x80\x03\x00\x01\x011'
b'TCSY\x07\x07\x00\x01\x00\xbb'
b'TCSY\x80\x05\x00\x06\x001'
b'\x00y\x00\x00\xe1TCSY\x85'
b'\x02\x00\x01R\x86TCSY\x80'
b'\x03\x00\x01\x011TCSY\x07'
b'\x07\x00\x01\x01\xbcTCSY\x80'
b'\x04\x00\x02\x00\x83\xb5TCSY'
b'\x81\x02\x00\x01\x10@TCSY'
b'\x80\x03\x00\x01\x011TCSY'
b'\x07\x07\x00\x01\x00\xbbTCSY'
b'\x80\x05\x00\x06\x001\x00y\x00\x00'


KeyboardInterrupt: 

# Serial Read Example #5 - pySerial V3

- YES! Finally works appropriately

FRAME HEADER = 5359

END OF FRAME = 5443 

In [2]:
import time
import serial

while True:
    serBarCode = serial.Serial('/dev/ttyUSB0', 115200, timeout=2)
    #read data from serial port
    #serBarCode.flushInput()
    #dataBarCode = serBarCode.read(12) # read 12 bytes at a time
    #dataBarCode = serBarCode.read()
    
    ch = serBarCode.read(12).hex()
    print (ch)
    # 12 bytes = max frame size for radar module
    #dataBarCode = serBarCode.readline()

    #if there is smth do smth
#     if len(str(dataBarCode.decode('hex'))) >= 1:
#         print(str(dataBarCode.decode('hex')))
    serBarCode.close()
    time.sleep(0.01)

005359800400020023555443
5359810200010d3d54435359
535980040002002355544353
535980050006000300220000
535985020001538754435359
535980040002002355544353
5359800300012d5d54435359
535980030001346454435359
535985020001538754435359
5359800500068011001e0000
5359810200010d3d54435359
535980040002002355544353
535980050006800f001f0000
535985020001538754435359
535980040002002355544353
535980030001073754435359
535980030001487854435359
535985020001538754435359
53590707000101bc54435359
5359810200010d3d54435359
53590707000100bb54435359
535980030001043454435359
535985020001538754435359
53590707000101bc54435359
5359810200010d3d54435359
5359800500068003002e0000
535980030001053554435359
535980040002002f61544353
535980030001063654435359
5359800500068005002e0000
5359810200010d3d54435359
53590707000100bb54435359
5359800300010b3b54435359
5359800500068008002e0000
5359850200014a7e54435359
535980040002002f61544353
5359800300010c3c54435359
535980030001104054435359
535985020001477b54435359
535980030001114154435359


KeyboardInterrupt: 

In [1]:
# Test with reading 15 Bytes

import time
import serial

while True:
    # Read data from serial port
    serBarCode = serial.Serial('/dev/ttyUSB0', 115200, timeout=2)
    
    ch = serBarCode.read(15).hex()
    print (ch)
    
    serBarCode.close()
    time.sleep(0.01)

00005359810200010d3d5443535980
535980040002005385544353598003
53590707000100bb54435359800500
5359850200014a7e54435359800200
535980030001033354435359070700
535980040002005385544353598102
535980030001013154435359800500
535980030001013154435359070700
535980040002005385544353598502
535980030001013154435359800500
5359810200010c3c54435359800300
535980040002005385544353598003
535980050006002b00460000a85443
535985020001447854435359800300
53590707000101bc54435359800400
5359810200010c3c54435359800300
535980050006000700520000905443
535980030001013154435359070700
535980040002005385544353598502
535980020001023154435359800300
53590707000101bc54435359800500
5359810200010c3c54435359800300
535980040002005385544353598002
535980030001033354435359800500
535985020001437754435359800200
535980030001033354435359800400
5359810200010d3d54435359800200
535980030001013154435359800500
535980030001013154435359800400
535985020001437754435359800200
535980030001093954435359800500
5359810200010d3d54435359800300
53598004

KeyboardInterrupt: 